# Simulation Results

This notebook summarizes the results of the optimization experiments.

In [1]:
import json
import os

import numpy as np
import pandas as pd

from IPython.display import display, HTML

In [2]:
def load_meta(**params):
    """Return metafile based on simulation properties."""
    metafn = os.path.join(params['base_dirn'],
                          params['metafn_fmt'].format(**params))
    return json.load(open(metafn, 'r'))


def write_results(results_df, **params):
    """Write simulation results summary."""
    formatters = params.get('formatters')
    sumcsvfn = os.path.join(params['base_dirn'], params['sumcsvfn'])
    results_df.to_csv(sumcsvfn, index=False)
    sumtexfn = os.path.join(params['base_dirn'], params['sumtexfn'])
    results_df.to_latex(sumtexfn, index=False, formatters=formatters)


def sim_results(**params):
    """Summarize simulation results for an algo-func combination."""
    meta = load_meta(**params)
    algstr = params['alg'].replace('_',' ').title()
    algstr = algstr if len(algstr) > 4 else algstr.upper()
    funcstr = params['func'].replace('_',' ').title()
    expxkmin, fxk = meta['exp_fxkmin'], np.array(meta['f(xk)'])
    delta_fxk = fxk[~np.isnan(fxk)] - expxkmin
    nsteps = np.array(meta['nsteps'])
    fxk_thresh = params.get('fxk_thresh', 1.)
    foundmin = np.fabs(delta_fxk) < fxk_thresh
    elapsed_sec = np.array(meta['elapsed_sec'])
    algmeta = [('nx0','n'),('T0','$T_0$'),
               ('alpha','$\\alpha$'),('tol','tol')]
    algmetastr = ' '.join(['{0}={1}'.format(n2, meta[n1])
                           for n1, n2 in algmeta if n1 in meta])
    return dict(
        alg=algstr,
        func=funcstr,
        ntrials=meta['ntrials'],
        nmin=sum(foundmin),
        mnit=np.mean(nsteps[~np.isnan(fxk)]),
        mae=np.sum(np.fabs(delta_fxk))/len(delta_fxk),
        msec=np.mean(elapsed_sec),
        algparm=algmetastr,
    )

In [3]:
opts = {
    'base_dirn': './sims/',
    'savefn_fmt': '{alg}-{func}-steps-{trial:02d}.npy',
    'metafn_fmt': '{alg}-{func}-meta.json',
    'sumcsvfn': 'results-sum.csv',
    'sumtexfn': 'results-sum.tex',
}

params = [
    {
        'alg': 'gradient_descent',
        'func': 'rosenbrock',
    },
    {
        'alg': 'gradient_descent',
        'func': 'goldstein_price',
        'fxk_thresh': 10,
    },
    {
        'alg': 'bfgs',
        'func': 'rosenbrock',
    },
    {
        'alg': 'bfgs',
        'func': 'goldstein_price',
        'fxk_thresh': 10,
    },
    {
        'alg': 'simulated_annealing',
        'func': 'rosenbrock',
    },
    {
        'alg': 'simulated_annealing',
        'func': 'goldstein_price',
        'fxk_thresh': 10,
    },
    {
        'alg': 'simulated_annealing',
        'func': 'bartels_conn',
    },
    {
        'alg': 'simulated_annealing',
        'func': 'egg_crate',
    },
    {
        'alg': 'particle_swarm',
        'func': 'rosenbrock',
    },
    {
        'alg': 'particle_swarm',
        'func': 'goldstein_price',
        'fxk_thresh': 10,
    },
    {
        'alg': 'particle_swarm',
        'func': 'bartels_conn',
    },
    {
        'alg': 'particle_swarm',
        'func': 'egg_crate',
    },
]

results_df = pd.DataFrame()
for param in params:
    param.update(**opts)
    results = sim_results(**param)
    results_df = results_df.append(results, ignore_index=True)

results_df = results_df.astype({'ntrials':int,'nmin':int,'mnit':int})
disp_cols = [
    'alg',
    'func',
    'ntrials',
    'nmin',
    'mae',
    'mnit',
    'msec',
    'algparm',
]

results_df = results_df[disp_cols].sort_values(by=['alg'])
formatters = {'mae':'{:,.2e}'.format,'msec':'{:,.2f}'.format}

write_results(results_df, **opts, formatters=formatters)

display(HTML(results_df.to_html(index=False, formatters=formatters)))

alg,func,ntrials,nmin,mae,mnit,msec,algparm
BFGS,Rosenbrock,12,12,2.51e-06,28,0.38,tol=0.01
BFGS,Goldstein Price,12,6,1.49e+02,16,0.58,tol=0.01
Gradient Descent,Rosenbrock,12,12,1.23e-04,8281,11.58,$\alpha$=0.001 tol=0.01
Gradient Descent,Goldstein Price,12,3,2.04e+02,2113,25.45,$\alpha$=1e-05 tol=0.01
Particle Swarm,Rosenbrock,12,11,7.21e-01,200,0.06,n=4
Particle Swarm,Goldstein Price,12,10,1.39e+01,2000,0.69,n=4
Particle Swarm,Bartels Conn,12,11,8.31e-01,300,0.20,n=3
Particle Swarm,Egg Crate,12,12,2.93e-01,2000,0.87,n=4
Simulated Annealing,Rosenbrock,12,12,2.21e-01,200,0.20,$T_0$=1.0
Simulated Annealing,Goldstein Price,12,6,4.36e+01,2000,1.17,$T_0$=1.0
